## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.insert(0, '..')

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
from typing import Optional
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [4]:
from openai import OpenAI
from together import Together

client = Together(
    api_key="95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
)

In [5]:

completion = client.chat.completions.create(
  extra_body={},
  model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ],
  stream=True,
)
for chunk in completion:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

The meaning of life is a question that has puzzled philosophers, theologians, scientists, and many others for centuries. It's a complex and subjective topic that can be approached from various perspectives. Here are some possible interpretations:

1. **Biological perspective**: From a biological standpoint, the meaning of life could be to survive and reproduce, ensuring the continuation of one's genes and species.
2. **Philosophical perspective**: Philosophers have offered various answers, such as:
	* **Hedonism**: The pursuit of pleasure and happiness.
	* **Eudaimonia**: Living a virtuous and fulfilling life, as advocated by Aristotle.
	* **Existentialism**: Creating one's own meaning and purpose in life, as proposed by Jean-Paul Sartre and Martin Heidegger.
3. **Religious perspective**: Many religions offer a sense of purpose and meaning, such as:
	* **Christianity**: Living a life of faith, love, and service to others, with the ultimate goal of reunification with God.
	* **Buddhism*

## Load dataset

In [7]:
df = pd.read_csv('../data/data_version1/reason_specialist - nhi khoa.csv', index_col=0)
df.head()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction
1480,259,5,2,0.0,56,0,nóng người chảy máu cam,nhi khoa,1
0,9,5,2,NaN,10,8,viêm a mãn tính,nhi khoa,1
1478,459,5,2,0.0,79,0,đau bụng vào buổi sáng,nhi khoa,1
2,9,5,2,NaN,-1,6,đi khám kl bị đau dạ dày,nhi khoa,1
1476,8,5,2,0.0,1,0,"con bị khó ngủ, chậm tăng cân, đổ nhiều mồ hôi...",nhi khoa,1


In [59]:
template = """You are a medical text transformation system that converts examination reasons into user questions ONLY if they contain actual symptoms or diseases.

# Rules:
1. Convert examination reasons to questions about which specialty to visit ONLY if they contain actual symptoms or diseases
2. CORRECT ANY SPELLING ERRORS OR TYPOS in the symptoms/diseases before using them in questions
3. Subject handling:
   - If input contains "cháu" → keep "Cháu bị [symptoms]" in the output
   - If specialist is "nhi khoa" but no subject is mentioned → use "Con tôi bị [symptoms]"
   - For all other cases → use "Tôi bị [symptoms]"
4. Format: "[Subject] bị [corrected symptoms/disease] thì nên đi khám khoa nào?"
5. Return null for ANY of these cases:
   - No symptoms (e.g., routine checkup, administrative procedures)
   - Past surgical procedures without symptoms (e.g., "đã mổ...")
   - Medical history without current symptoms
   - Any non-symptom or non-disease information
6. Keep all actual symptoms/diseases from the original reason (but with corrected spelling)
7. NEVER mention the specialty name in the question

# Types of text to return null for:
- Surgical history without symptoms
- Routine checkups
- Follow-up appointments
- Administrative requests
- Past medical procedures without current symptoms
- Medical record requests
- Test results without symptoms

# Examples:

## Example 1 (Valid - pediatric case):
Reason: nóng người chảy máu cam
Specialist: nhi khoa
Output: {"user_question": "Con tôi bị nóng người chảy máu cam thì nên đi khám khoa nào?"}

## Example 2 (Valid - adult case):
Reason: đau đầu, chóng mặt, buồn nôn
Specialist: thần kinh
Output: {"user_question": "Tôi bị đau đầu, chóng mặt, buồn nôn thì nên đi khám khoa nào?"}

## Example 3 (Invalid - no symptoms):
Reason: khám sức khỏe định kỳ
Specialist: khoa khám bệnh
Output: {"user_question": null}

## Example 4 (Valid - specific subject "cháu"):
Reason: cháu mới có biểu hiện nháy mắt
Specialist: nhi khoa
Output: {"user_question": "Cháu bị nháy mắt thì nên đi khám khoa nào?"}

## Example 5 (Invalid - past procedure):
Reason: đã mổ hẹp khúc nối bể thận
Specialist: nhi khoa
Output: {"user_question": null}

## Example 6 (Valid - with spelling error):
Reason: đau dat dày, ợ chua
Specialist: tiêu hóa
Output: {"user_question": "Tôi bị đau dạ dày, ợ chua thì nên đi khám khoa nào?"}

## Example 7 (Valid - with typo):
Reason: sôt cao, đao họng
Specialist: tai mũi họng
Output: {"user_question": "Tôi bị sốt cao, đau họng thì nên đi khám khoa nào?"}

# Response Format:
Respond in JSON format with a single field:
{
    "user_question": "The question or null if no symptoms/diseases are present"
}"""

In [60]:
import json

@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=1, max=60))
def generate_question(reason: str, specialist: str) -> Optional[str]:
    user_prompt = f"Reason: {reason}\nSpecialist: {specialist}"
    try:
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
            messages=[
                {"role": "system", "content": template},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        print('result after generate: ', result)
        return result.get('user_question')
        
    except Exception as e:
        print(f"Error generating question: {e}")
        return None


In [61]:
test = generate_question(
    reason="đau đầu, chóng mặt, buồn nôn",
    specialist="thần kinh"
)

result after generate:  {'user_question': 'Tôi bị đau đầu, chóng mặt, buồn nôn thì nên đi khám khoa nào?'}


In [62]:
test

'Tôi bị đau đầu, chóng mặt, buồn nôn thì nên đi khám khoa nào?'

In [1]:
# Process first 1000 rows from the dataset
from tqdm import tqdm

sample_df = df.head(1000).copy()
sample_df['user_question'] = None

# Apply the generate_question function to each row using a for loop with tqdm
for idx in tqdm(sample_df.index, desc="Generating questions"):
    reason = sample_df.loc[idx, 'reason_combind']
    specialist = sample_df.loc[idx, 'specialist_name']
    question = generate_question(reason=reason, specialist=specialist)
    sample_df.loc[idx, 'user_question'] = question
    # Print the generated question for visibility
    if question:
        print(f"Input: {reason} | Output: {question}")

# Display the results
print(f"Generated questions for {sample_df['user_question'].notna().sum()} out of 1000 rows")
print(f"Number of null questions: {sample_df['user_question'].isna().sum()}")

# Save the processed data
sample_df.to_csv('../data/processed_questions_sample.csv', index=True)
sample_df.head()

NameError: name 'df' is not defined

## enhance reason scheduler

In [6]:
system_prompt = """You are a medical text transformation system that converts examination reasons into symptoms and diseases ONLY when they contain actual symptoms or diseases.

# Rules:
1. Convert examination reasons to symptoms and diseases ONLY if they contain actual medical conditions
2. Correct any spelling errors or typos in the symptoms/diseases before formatting
3. Format output as: "symptom1, symptom2, disease1, disease2" with all items separated by commas
4. Return null for ANY of these cases:
   - If symptoms/diseases don't relate to the specialist name
   - No symptoms present (e.g., routine checkup, administrative procedures) EXCEPT for "nội khoa" specialist
   - Past surgical procedures without current symptoms (e.g., "đã mổ...")
   - Medical history without current symptoms
   - Any non-symptom or non-disease information
5. Special case: For "nội khoa" (internal medicine) specialist, accept examination reasons like "khám tổng quát" (general checkup) and return them as is

# Types of text to return null for:
- Surgical history without symptoms
- Routine checkups (except for "nội khoa" specialist)
- Follow-up appointments
- Administrative requests
- Past medical procedures without current symptoms
- Medical record requests
- Test results without symptoms

# Response Format:
Respond in JSON format with a single field:
{
    "result": "The symptoms/diseases processed or null if no symptoms/diseases are present"
}
"""

In [7]:
reason = "khám tổng quát"
specialist = "nội khoa"

In [8]:
user_prompt = f"Reason: {reason}\nSpecialist: {specialist}"

completion = client.chat.completions.create(
  extra_body={},
  model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
  messages=[
    {
        "role": "system",
        "content": system_prompt
    },
    {
      "role": "user",
      "content": user_prompt
    }
  ],
  stream=True,
)
for chunk in completion:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

{
    "result": "khám tổng quát"
}


In [9]:
import json

@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=1, max=60))
def reason_handle(client, reason: str, specialist: str) -> Optional[str]:
    user_prompt = f"Reason: {reason}\nSpecialist: {specialist}"
    try:
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        print('result after generate: ', result)
        return result.get('result')
        
    except Exception as e:
        print(f"Error generating question: {e}")
        return None

In [11]:
# Function to apply generate_question to the entire DataFrame
def process_dataframe(client, df, start_idx=0, end_idx=None, output_path='../data/processed_questions.csv'):
    """
    Process the dataframe by applying generate_question function to each row
    
    Args:
        client: API client instance
        df (DataFrame): Input dataframe
        start_idx (int): Starting index to process from
        end_idx (int): Ending index (exclusive), if None process until the end
        output_path (str): Path to save the processed dataframe
    
    Returns:
        DataFrame: The processed dataframe with user_question column
    """
    # Make a copy of the dataframe or slice it if needed
    if end_idx is None:
        processed_df = df.iloc[start_idx:].copy()
    else:
        processed_df = df.iloc[start_idx:end_idx].copy()
    
    # Initialize symptoms/diseases column if it doesn't exist
    if 'symptoms/diseases' not in processed_df.columns:
        processed_df['symptoms/diseases'] = None
    
    # Apply the generate_question function to each row using a for loop with tqdm
    for idx in tqdm(processed_df.index, desc="Handle reason combind"):
        reason = processed_df.loc[idx, 'reason_combind']
        specialist = processed_df.loc[idx, 'specialist_name']
        result = reason_handle(client, reason=reason, specialist=specialist)
        processed_df.loc[idx, 'symptoms/diseases'] = result
        # Print the generated question for visibility
        if result:
            print(f"Input: {reason} | {specialist} | Output: {result}")
        
        # Save after every 500 records
        if (idx - processed_df.index[0] + 1) % 500 == 0:
            interim_path = output_path.replace('.csv', f'_interim_{idx}.csv')
            processed_df.to_csv(interim_path, index=True)
            print(f"Saved interim results at index {idx} to {interim_path}")
    
    # Display the results
    print(f"Generated questions for {processed_df['symptoms/diseases'].notna().sum()} out of {len(processed_df)} rows")
    print(f"Number of null questions: {processed_df['symptoms/diseases'].isna().sum()}")
    
    # Save the final processed data
    processed_df.to_csv(output_path, index=True)
    
    return processed_df


In [12]:
nhi_khoa_df = pd.read_csv('../data/data_version1/reason_specialist - nhi khoa.csv', index_col=0)

In [29]:
nhi_khoa_df.head()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction
1480,259,5,2,0.0,56,0,nóng người chảy máu cam,nhi khoa,1
0,9,5,2,NaN,10,8,viêm a mãn tính,nhi khoa,1
1478,459,5,2,0.0,79,0,đau bụng vào buổi sáng,nhi khoa,1
2,9,5,2,NaN,-1,6,đi khám kl bị đau dạ dày,nhi khoa,1
1476,8,5,2,0.0,1,0,"con bị khó ngủ, chậm tăng cân, đổ nhiều mồ hôi...",nhi khoa,1


In [ ]:
nhi_khoa_df = process_dataframe(nhi_khoa_df, output_path='../data/data_version2/nhi_khoa.csv')

In [34]:
san_phu_khoa = pd.read_csv('../data/data_version1/reason_specialist - sản phụ khoa.csv', index_col=0)

In [35]:
san_phu_khoa.shape

(1603, 8)

In [ ]:
san_phu_khoa_df = process_dataframe(san_phu_khoa, output_path="../data/data_version2/san_phu_khoa.csv")

Handle reason combind:   0%|                                                                                                                            | 1/1603 [00:01<47:33,  1.78s/it]

result after generate:  {'result': None}


Handle reason combind:   0%|▏                                                                                                                         | 2/1603 [00:10<2:32:38,  5.72s/it]

result after generate:  {'result': 'null'}
Input: khám phụ khoa | sản phụ khoa | Output: null


Handle reason combind:   0%|▏                                                                                                                         | 3/1603 [00:14<2:13:43,  5.01s/it]

...

result after generate:  {'result': 'polyp, ctc'}
Input: polyp ctc | sản phụ khoa | Output: polyp, ctc


Handle reason combind:  84%|████████████████████████████████████████████████████████████████████████████████████████████████████▍                  | 1353/1603 [3:04:53<40:14,  9.66s/it]

result after generate:  {'result': 'rối loạn chu kỳ kinh nguyệt, kinh nguyệt ngắn ngày'}
Input: khám tầm soát, bị rối loại chu kỳ kinh nguyệt, kinh nguyệt ngắn ngày | sản phụ khoa | Output: rối loạn chu kỳ kinh nguyệt, kinh nguyệt ngắn ngày


Handle reason combind:  84%|████████████████████████████████████████████████████████████████████████████████████████████████████▌                  | 1354/1603 [3:04:55<31:04,  7.49s/it]

result after generate:  {'result': 'dịch bất thường vùng kín, mùi khó chịu'}
Input: vùng kín có dịch bất thường, kèm mùi | sản phụ khoa | Output: dịch bất thường vùng kín, mùi khó chịu


In [ ]:
import threading
import concurrent.futures

def process_data_thread(api_key: str, df, output_path: str, thread_name: str):
    print(f"Starting {thread_name}")
    client = Together(api_key=api_key)
    result_df = process_dataframe(client, df, output_path=output_path)
    print(f"Completed {thread_name}")
    return result_df

# Load the datasets
tai_mui_hong = pd.read_csv('../data/data_version1/reason_specialist - tai mũi họng.csv', index_col=0)
noi_khoa = pd.read_csv('../data/data_version1/reason_specialist - nội khoa.csv', index_col=0)

# API keys
api_key1 = "95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
api_key2 = "984a9676fb29b00a470440df6d9b93e03a9b35a345f3a324bbeb5fb7a9f46c84"

# Create and start threads
thread_1 = threading.Thread(
    target=process_data_thread, 
    args=(api_key1, tai_mui_hong, '../data/data_version2/tai_mui_hong.csv', 'Tai mui hong thread')
)

thread_2 = threading.Thread(
    target=process_data_thread, 
    args=(api_key2, noi_khoa, '../data/data_version2/noi_khoa.csv', 'Noi khoa thread')
)

# Start threads
thread_1.start()
thread_2.start()

# Wait for threads to complete
thread_1.join()
thread_2.join()

print("All threads completed")

Starting Tai mui hong thread
Starting Noi khoa thread


Handle reason combind:   0%|          | 0/4106 [00:00<?, ?it/s]

Handle reason combind:   0%|          | 0/4556 [00:00<?, ?it/s]

result after generate:  {'result': 'viêm mũi dị ứng'}
Input: viêm mũi dị ứng | tai mũi họng | Output: viêm mũi dị ứng
result after generate:  {'result': 'ho, viêm phổi'}
Input: ho ,viêm phổi | nội khoa | Output: ho, viêm phổi
result after generate:  {'result': 'đau thân, nổi hạch, đau lưng'}
Input: đau thân,nổi hạch, đau lưng | nội khoa | Output: đau thân, nổi hạch, đau lưng
result after generate:  {'result': 'viêm họng, ho kéo dài'}
Input: viêm họng,ho kéo dài | tai mũi họng | Output: viêm họng, ho kéo dài
result after generate:  {'result': 'khám tổng quát'}
Input: muốn khám nội tổng quát | nội khoa | Output: khám tổng quát
result after generate:  {'result': 'đi đái màu nước coca'}
Input: đi đái màu nước coca | nội khoa | Output: đi đái màu nước coca
result after generate:  {'result': 'sổ mũi, hay quên'}
Input: sổ mũi ,hay quên | tai mũi họng | Output: sổ mũi, hay quên
result after generate:  {'result': 'đau họng, ít đờm'}
Input: đau họng,ít đờm | tai m

In [ ]:
rối loạn kinh nguyệt kéo dài